In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Capture video via webcam

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Media Pipe feed", frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
for i in range (1,5):
    cv2.waitKey(1)

### 1. Make Detections

In [ ]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("./data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

### 2. Determine joints

In [4]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("./data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            print("No landmarks found")
            pass

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[x: 0.37284485
y: 0.19525093
z: -0.13823296
visibility: 0.9998889
, x: 0.39590713
y: 0.16961354
z: -0.20154434
visibility: 0.9999008
, x: 0.4025015
y: 0.17025506
z: -0.20174149
visibility: 0.999912
, x: 0.40981686
y: 0.17074561
z: -0.20217773
visibility: 0.99990344
, x: 0.38692528
y: 0.16775352
z: -0.115793474
visibility: 0.999895
, x: 0.3872813
y: 0.16732627
z: -0.11559655
visibility: 0.9999063
, x: 0.3878811
y: 0.16677022
z: -0.115761966
visibility: 0.99991274
, x: 0.46440363
y: 0.18154186
z: -0.34592536
visibility: 0.9999721
, x: 0.43289328
y: 0.17859757
z: 0.028863005
visibility: 0.99995863
, x: 0.39438057
y: 0.22807294
z: -0.18103352
visibility: 0.9999548
, x: 0.38251668
y: 0.22605735
z: -0.072260834
visibility: 0.99992895
, x: 0.5421179
y: 0.349927
z: -0.46621224
visibility: 0.99946564
, x: 0.48727876
y: 0.33885887
z: 0.27706364
visibility: 0.99926466
, x: 0.515861
y: 0.58110845
z: -0.30216947
visibility: 0.9598313
, x: 0.46777296
y: 0.5301287
z: 0.26411518
visibility: 0.14325973

In [18]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.55278516
y: 0.35183862
z: -0.6540659
visibility: 0.9998903

In [19]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]


x: 0.4054467
y: 0.7724296
z: -0.57371765
visibility: 0.9798264

In [24]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]


x: 0.53770936
y: 0.6176323
z: -0.6611603
visibility: 0.9961536

### 3. Calculate Angles

In [3]:
def calculate_angles(point1: list, point2: list, point3: list) -> float:
    point1 = np.array(point1)
    point2 = np.array(point2)
    point3 = np.array(point3)

    # Calculate degree algo
    angleInRad = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angleInDeg = np.abs(angleInRad * 180.0 / np.pi)

    angleInDeg = angleInDeg if angleInDeg <= 180 else 360 - angleInDeg
    return angleInDeg


def calculate_angles_2(v1, v2) -> float:
    dot = np.dot(v1, v2)
    mod_v1 = np.linalg.norm(v1)
    mod_v2 = np.linalg.norm(v2)
    cos_theta = dot/(mod_v1*mod_v2)
    theta = math.acos(cos_theta)

    angleInDeg = np.abs(theta * 180.0 / np.pi)
    angleInDeg = angleInDeg if angleInDeg <= 180 else 360 - angleInDeg
    return angleInDeg

In [4]:
shoulder = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z ])
elbow = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z ])
wrist = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z ])

NameError: name 'landmarks' is not defined

In [10]:
shoulder, elbow, wrist

(array([ 0.5289976 ,  0.34566823, -0.43016756]),
 array([ 0.48711345,  0.55861193, -0.36767742]),
 array([0.34201512, 0.43823504, 0.05330645]))

In [18]:
calculate_angles(shoulder, elbow, wrist)

61.44765364410915

In [19]:
calculate_angles_2(wrist - elbow, shoulder - elbow)

93.71573952238882

In [25]:
from numpy.lib.function_base import angle


VIDEO_DIM = [946, 1088]

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("./data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get joints' coordinates
            # shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
            # elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            # wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

            # angle = calculate_angles(shoulder, elbow, wrist)

            shoulder = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z ])
            elbow = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z ])
            wrist = np.array([ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z ])

            angle = calculate_angles_2(wrist - elbow, shoulder - elbow)

            # Visualize
            cv2.putText(image, str(angle), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(e)
            pass

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

error: OpenCV(4.6.0) /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 

### 4. Counter

In [48]:
VIDEO_DIM = [946, 1088]
stage = "down"
counter = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("./data/db_curl_2.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get joints' coordinates
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

            angle = calculate_angles(shoulder, elbow, wrist)

            # Curl counter
            if angle > 120:
                stage = "down"
            elif angle < 60 and stage == "down":
                stage = "up"
                counter += 1

            # Visualize
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, VIDEO_DIM).astype(int)), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(e)
            pass

        # Visualize Counter
        cv2.putText(image, str(counter), (50, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)